In [ ]:
!pip install hazm
!pip install python-docx
!pip install rake_nltk
!pip install docx
!pip install stanza

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 892.6/892.6 kB 11.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.3/17.3 MB 70.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.4/4.4 MB 82.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 52.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 243.3/243.3 kB 16.8 MB/s eta 0:00:00
  Created wheel for flashtext: filename=flashtext-2.7-py2.py3-none-any.whl size=9298 sha256=d13c53d01dbf259ab0b8150d7897e4921d756ef78ecc5ec1dc1e448403d51c25
  Stored in directory: /root/.cache/pip/wheels/49/20/47/f03dfa8a7239c54cbc44ff7389eefbf888d2c1873edaaec888
Successfully built flashtext
  Attempting uninstall: numpy
    Found existing installation: numpy 1.26.4
    Uninstalling numpy-1.26.4:
      Successfully uninstalled numpy-1.26.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviou

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.3/244.3 kB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.9/54.9 kB 2.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for docx: filename=docx-0.2.4-py3-none-any.whl size=53892 sha256=17311ed469885ba4e6052417032ef758d6d9423b42d9764e98f2340ad51dd192
  Stored in directory: /root/.cache/pip/wheels/c1/3e/c3/e81c11effd0be5658a035947c66792dd993bcff317eae0e1ed
Successfully built docx
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 14.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 72.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 60.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 40.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21

In [64]:
import numpy as np
import pandas as pd
from hazm import stopwords_list, Normalizer, WordTokenizer, SentenceTokenizer, Stemmer, Lemmatizer, sent_tokenize, word_tokenize
import docx
import re
from sklearn.feature_extraction.text import TfidfVectorizer
import requests
from sklearn.metrics.pairwise import cosine_similarity
import openai
import nltk
from rake_nltk import Rake
from google.colab import drive
import stanza
from collections import defaultdict

In [ ]:
stanza.download('fa')
nlp = stanza.Pipeline('fa')

INFO:stanza:Downloaded file to /root/stanza_resources/resources.json
INFO:stanza:Downloading default packages for language: fa (Persian) ...


INFO:stanza:Downloaded file to /root/stanza_resources/fa/default.zip
INFO:stanza:Finished downloading models and saved to /root/stanza_resources
INFO:stanza:Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES


INFO:stanza:Downloaded file to /root/stanza_resources/resources.json
INFO:stanza:Loading these models for language: fa (Persian):
| Processor | Package        |
------------------------------
| tokenize  | perdt          |
| mwt       | perdt          |
| pos       | perdt_charlm   |
| lemma     | perdt_nocharlm |
| depparse  | perdt_charlm   |
| ner       | arman          |

INFO:stanza:Using device: cpu
INFO:stanza:Loading: tokenize
INFO:stanza:Loading: mwt
INFO:stanza:Loading: pos
INFO:stanza:Loading: lemma
INFO:stanza:Loading: depparse
INFO:stanza:Loading: ner
INFO:stanza:Done loading processors!


In [ ]:
class PersianRAKE(Rake):
    def _tokenize_text_to_sentences(self, text: str):
        return sent_tokenize(text)

    def _tokenize_sentence_to_words(self, sentence: str):
        return word_tokenize(sentence)

In [66]:
def read_from_docx(doc):
    fullText=''
    for pra in doc.paragraphs:
        fullText+=pra.text+' '

    return fullText

def split_into_overlapping_chunks(sentences, max_chunk_size=1000, overlap_size=200):
    chunks = []
    current_chunk = ""
    current_chunk_size = 0

    for sentence in sentences:
        sentence_length = len(sentence)

        if current_chunk_size + sentence_length > max_chunk_size and current_chunk:
            chunks.append(current_chunk.strip())

            overlap_buffer = current_chunk[-overlap_size:].strip() if current_chunk else ""
            current_chunk = overlap_buffer + " "
            current_chunk_size = len(overlap_buffer) + 1

        current_chunk += sentence + " "
        current_chunk_size += sentence_length + 1

    if current_chunk:
        chunks.append(current_chunk.strip())

    return chunks

def preprocess_text_1(text):
    # text = text.lower()
    text = re.sub(r'[^\w\s]', '', text)
    text = re.sub(r'( +)', ' ', str(text))
    return text


def preprocess_text_2(text):
    text = re.sub('(\(.*?\))|(\[.*?\])', '', str(text))
    text = re.sub(r'( +)', ' ', str(text))

    word_tokenizer = WordTokenizer()
    words = word_tokenizer.tokenize(text)

    stopwords = stopwords_list()
    filtered_words = [word for word in words if word not in stopwords]

    lemmatizer = Lemmatizer()
    lemmatized_words = [lemmatizer.lemmatize(word) for word in filtered_words]

    return ' '.join(lemmatized_words)


def check_spelling(main_text):
    endpoint = "https://api.languagetool.org/v2/check"

    data = {
        "text": main_text,
        "language": "en-US",
    }

    response = requests.post(endpoint, data=data)
    json_response = response.json()

    updated_text = main_text

    for match in json_response.get("matches", []):
        replacement = match["replacements"][0]["value"] if match["replacements"] else ""

        offset = match["offset"]
        length = match["length"]

        updated_text = updated_text.replace(main_text[offset:offset+length], replacement)

    print("Original Query: ", main_text)
    print("Spell-checked Query: ", updated_text)
    return updated_text


def extract_key_phrases(query, max_phrases, lang="persian"):
    persian_stopwords = stopwords_list()
    r = PersianRAKE(
        stopwords=persian_stopwords,
        max_length= 3
    )

    r.extract_keywords_from_text(query)
    phrases = r.get_ranked_phrases()

    print("Key Phrases:", phrases[:2])

    multi_word_phrases = [p for p in phrases if len(p.split()) > 1]

    return multi_word_phrases[:max_phrases]

def phrase_search(sentence):
    doc = nlp(sentence)

    phrases = []
    for sent in doc.sentences:
        for word in sent.words:
            if word.upos in ['NOUN', 'ADJ']:
                phrase = word.text
                for other_word in sent.words:
                    if other_word.head == word.id and other_word.upos in ['NOUN', 'ADJ']:
                        phrase += " " + other_word.text
                if " " in phrase:
                    phrases.append(phrase)
    return phrases

def english_to_persian_number(number_str):
    english_to_persian = {
        "0": "۰",
        "1": "۱",
        "2": "۲",
        "3": "۳",
        "4": "۴",
        "5": "۵",
        "6": "۶",
        "7": "۷",
        "8": "۸",
        "9": "۹",
    }
    persian_number = "".join([english_to_persian[digit] for digit in number_str])
    return persian_number

def persian_words_to_number(sentence):
    word_to_number = {
        "صفر": 0,
        "یک": 1,
        "دو": 2,
        "سه": 3,
        "چهار": 4,
        "پنج": 5,
        "شش": 6,
        "هفت": 7,
        "هشت": 8,
        "نه": 9,
        "ده": 10,
        "یازده": 11,
        "دوازده": 12,
        "سیزده": 13,
        "چهارده": 14,
        "پانزده": 15,
        "شانزده": 16,
        "هفده": 17,
        "هجده": 18,
        "نوزده": 19,
        "بیست": 20,
        "سی": 30,
        "چهل": 40,
        "پنجاه": 50,
        "شصت": 60,
        "هفتاد": 70,
        "هشتاد": 80,
        "نود": 90,
        "صد": 100,
        "یکصد": 100,
        "دویست": 200,
        "سیصد": 300,
        "چهارصد": 400,
        "پانصد": 500,
        "ششصد": 600,
        "هفتصد": 700,
        "هشتصد": 800,
        "نهصد": 900,
        "هزار": 1000,
    }
    words = sentence.split(' ')

    result = []
    temp_number_words = []
    current_number = 0

    for word in words:
        if word[-1:]=='م' and (word[:-1] in word_to_number):
            word = word[:-1]
        if word in word_to_number:
            temp_number_words.append(word)
            current_number += word_to_number[word]
        else:
            if temp_number_words:
                english_number_str = str(current_number)
                persian_number_str = english_to_persian_number(english_number_str)
                result.append(persian_number_str)
                temp_number_words = []
                current_number = 0
            result.append(word)

    if temp_number_words:
        english_number_str = str(current_number)
        persian_number_str = english_to_persian_number(english_number_str)
        result.append(persian_number_str)

    return ' '.join(result)

def tfidf(docs, phrase):
    vectorizer = TfidfVectorizer(ngram_range=(1, 3))
    tfidf_matrix = vectorizer.fit_transform(docs)
    phrase_vector = vectorizer.transform(phrase)
    return cosine_similarity(phrase_vector, tfidf_matrix)

def preprocess_phrases(text, phrases):
    for phrase in phrases:
        text = text.replace(phrase, phrase.replace(" ", "_"))
    return text

def extract_whole_persian_numbers(text):
    persian_digits = "۰۱۲۳۴۵۶۷۸۹"
    return re.findall(f"[{persian_digits}]+", text)

def calculate_idf(document_numbers, numbers):
    idf = {}
    total_docs = len(document_numbers)
    for num in numbers:
        doc_count = sum(1 for doc in document_numbers if num in doc)
        idf[num] = np.log((total_docs + 1) / (doc_count + 1)) + 1
    return idf

def calculate_tf(document_numbers):
    tf = []
    for doc in document_numbers:
        tf_dict = defaultdict(int)
        for num in doc:
            tf_dict[num] += 1
        tf.append(tf_dict)
    return tf

def calculate_tf_idf(tf, idf):
    tf_idf = []
    for doc_tf in tf:
        doc_tf_idf = {}
        for num, freq in doc_tf.items():
            if num in idf:
                doc_tf_idf[num] = freq * idf[num]
        tf_idf.append(doc_tf_idf)
    return tf_idf

In [ ]:
# Loading The Main Document
drive.mount('/content/drive')
document = read_from_docx(docx.Document("/content/drive/My Drive/Constitution_of_the_Islamic_Republic.docx"))

Mounted at /content/drive


In [ ]:
# Chunking The Document
normalizer = Normalizer()
normalized_text = normalizer.normalize(document)

sentence_tokenizer = SentenceTokenizer()
sentences = sentence_tokenizer.tokenize(normalized_text)

max_chunk_size = 1000
overlap_size = 200
chunks = split_into_overlapping_chunks(sentences, max_chunk_size, overlap_size)
print("<Chunk 1>")
print(f"Original Chunk:\n{chunks[0]}")

# Preprocessing The Chunks
preprocessed1_chunks = [preprocess_text_1(chunk) for chunk in chunks]

preprocessed2_chunks = [preprocess_text_2(chunk) for chunk in preprocessed1_chunks]
print(f"Preprocessed Chunk:\n{preprocessed2_chunks[0]}")

<Chunk 1>
Original Chunk:
قانون اساسی جمهوری اسلامی ایران مبین نهادهای فرهنگی، اجتماعی، سیاسی و اقتصادی جامعه ایران بر اساس اصول و ضوابط اسلامی است که انعکاس خواست قلبی امت اسلامی می‏باشد. ماهیت انقلاب عظیم اسلامی ایران و روند مبارزه مردم مسلمان از ابتدا تا پیروزی که در شعارهای قاطع و کوبنده همه قشرهای مردم تبلور می‏یافت این خواست اساسی را مشخص کرده و اکنون در طلیعه این پیروزی بزرگ، ملت ما با تمام وجود نیل به آن را می‏طلبد. ویژگی بنیادی این انقلاب نسبت به دیگر نهضت‏های ایران در سده اخیر، مکتبی و اسلامی بودن آن است. ملت مسلمان ایران پس از گذر از نهضت ضد استبدادی مشروطه و نهضت ضد استعماری ملی‏ شدن نفت به این تجربه گرانبار دست یافت که علت اساسی و مشخص عدم موفقیت این نهضت‏ها مکتبی نبودن مبارزات بوده است. گرچه در نهضت‏های اخیر خط فکری اسلامی و رهبری روحانیت مبارز سهم اصلی و اساسی را بر عهده داشت، ولی به دلیل دور شدن این مبارزات از مواضع اصیل اسلامی، جنبش‏ها به سرعت به رکود کشانده شد.
Preprocessed Chunk:
قانون اساسی جمهوری اسلامی ایران مبین نهاد فرهیخت#فرهنگ اجتماعی سیاسی اقتصادی جامعه ایران

In [ ]:
# query = "وظایف قوه قضائیه چیست؟"
query = "اصل هفتم قانون اساسی جمهوری اسلامی درباره چیست؟"
query = check_spelling(query)

processed_query = preprocess_text_2(preprocess_text_1(query))
print(query, "\n", processed_query)

Original Query:  اصل هفتم قانون اساسی جمهوری اسلامی درباره چیست؟
Spell-checked Query:  اصل هفتم قانون اساسی جمهوری اسلامی درباره چیست؟
اصل هفتم قانون اساسی جمهوری اسلامی درباره چیست؟ 
 اصل هفتم قانون اساسی جمهوری اسلامی چیست


In [ ]:
query_n = persian_words_to_number(processed_query)
print(query_n)

اصل ۷ قانون اساسی جمهوری اسلامی چیست


In [ ]:
# این پایینی بعدا حتما چک شود(ممکنه تو جستجوی عبارت اشتباه پیش بیاد باید ضریب سرچ عبارت کاهش یابد)
# query = "اصل هفتم قانون اساسی جمهوری اساسی درباره چیست؟"

phrases = phrase_search(query_n)
print(phrases)

['اصل ۷', '۷ قانون', 'قانون اساسی']


In [ ]:
# doc = nlp(query_n)

numbers = re.findall(r'\d+', query_n)

print(numbers)

['۷']


In [ ]:
numberic_chunks = [persian_words_to_number(chunk) for chunk in preprocessed2_chunks]

In [67]:
# Cosin Similarity Of Query
cosine_sim_query = tfidf(numberic_chunks, [query_n]).flatten()


# Cosin Similarity Of Number
document_numbers = [extract_whole_persian_numbers(doc) for doc in numberic_chunks]
tf_numbers = calculate_tf(document_numbers)
idf_numbers = calculate_idf(document_numbers, numbers)
tf_idf_numbers = calculate_tf_idf(tf_numbers, idf_numbers)

number_bonus = np.zeros(len(numberic_chunks))
for i, doc_tf_idf in enumerate(tf_idf_numbers):
    for num in numbers:
        if num in doc_tf_idf:
            number_bonus[i] += doc_tf_idf[num]
if np.max(number_bonus) > 0:
    number_bonus = number_bonus / np.max(number_bonus)


# Cosin Similarity Of Phrase
preprocessed_docs = [preprocess_phrases(doc, phrases) for doc in numberic_chunks]
preprocessed_phrases = preprocess_phrases(query_n, phrases)

phrases_similarity = tfidf(preprocessed_docs, [preprocessed_phrases])
phrases_bonus = np.max(phrases_similarity, axis=0)
if np.max(phrases_bonus) > 0:
    phrases_bonus = phrases_bonus / np.max(phrases_bonus)


query_coef = 0.5
numbers_coef = 0.3
phrases_coef = 0.2

hybrid_scores = (cosine_sim_query * query_coef) + (number_bonus * numbers_coef) + (phrases_bonus * phrases_coef)

In [68]:
top_k = 3
indices = np.argsort(-hybrid_scores)[:top_k]

top_3 = [numberic_chunks[idx] for idx in indices]

print("Top 3 Hybrid Results:")
for idx, chunk in zip(indices, top_3):
    print(f"Chunk {idx + 1} (Score: {hybrid_scores[idx]:.2f}):\n{chunk}")
    print(f"{(cosine_sim_query * query_coef)[idx]:.2f} , {(number_bonus * numbers_coef)[idx]:.2f} , {(phrases_bonus * phrases_coef)[idx]:.2f}")

Top 3 Hybrid Results:
Chunk 65 (Score: 0.45):
دالت تقوا رهبری امت اسلام ۳ بینش صحیح سیاسی اجتماعی تدبیر شجاعت مدیریت قدرت رهبری صورت تعدد واجدین شرایط بینش فقه سیاسی قو مقدم اصل ۱۱۰ وظایف اختیارات رهبر ۱ سیاست نظام جمهوری اسلامی ایران مشورت مجمع تشخیص مصلحت نظام ۲ نظارت حسن اجرای سیاست نظام ۳ فرمان همهپرسی ۴ فرماندهی نیرو مسلح ۵ اعلان جنگ صلح بسیج نیرو ۶ نصب عزل قبول استعفاء الف فق شورای نگهبان ب عال مقام قوه قضائیه ج رئیس سازمان صدا سیما جمهوری اسلامی ایران د رئیس ستاد مشترک #هست فرمانده سپاه پاسدار انقلاب اسلامی فرمانده نیرو نظامی انتظامی ۷ اختلاف تنظیم روابط قوای سهگانه ۸ معضلات نظام طرق عادی قابلحل طریق مجمع تشخیص مصلحت نظام ۹ امضاء حکم ریاست جمهوری انتخاب مردم صلاحیت داوطلب ریاست جمهوری جهت دارا شرایط قانون میآید انتخابات تأیید شورای نگهبان دوره تأیید رهبری رسید#رس
0.02 , 0.30 , 0.13
Chunk 24 (Score: 0.43):
اتکاء آراء عمومی اداره انتخابات انتخاب رئیسجمهور نمایندگان مجلس شورای اسلامی اعضا شورا نظایر اینها همهپرسی موارد اصول قانون معین میگردد اصل ۷ دستور قرآن کریم أمرهم شوری بینه شا

In [ ]:
query_coef = 0.7
numbers_coef = 0.5
cosine_sim_query = tfidf(numberic_chunks, [query_n])
number_bonus = np.zeros(len(numberic_chunks))
if len(numbers)!=0:
    for i, chunk in enumerate(numberic_chunks):
      for number in numbers:
          if number in chunk:
              number_bonus[i] += 0.2
hybrid_scores = (cosine_sim_query * query_coef) + (number_bonus * numbers_coef)
# cosine_sim_phrase = tfidf(numberic_chunks, phrases)

# phrase_bonus = np.zeros(len(numberic_chunks))
# for i, chunk in enumerate(numberic_chunks):
#     for phrase in phrases:
#         if phrase in chunk:
#             for number in numbers:
#                 if number in phrase:
#                     phrase_bonus[i] += 1.0
#                 else:
#                     phrase_bonus[i] += 0.5
query_coef = 0
numbers_coef = 0.5
phrase_coef = 0

cosine_sim_query = tfidf(numberic_chunks, [query_n])

number_bonus = np.zeros(len(numberic_chunks))
if len(numbers)!=0:
    for i, chunk in enumerate(numberic_chunks):
      for number in numbers:
          if number in chunk:
              number_bonus[i] += 0.2



hybrid_scores = (cosine_sim_query * query_coef) + (number_bonus * numbers_coef) + (number_bonus * phrase_coef)

top_k = 3
indices = np.argsort(-hybrid_scores)[:top_k]

top_3 = [numberic_chunks[idx] for idx in indices]

print("Top 3 Hybrid Results:")
for idx, chunk in zip(indices, top_3):
    print(f"Chunk {idx + 1} (Score: {hybrid_scores[idx]:.2f}):\n{chunk}")
    print(f"{(cosine_sim_query * cosine_query_coef)[idx]:.2f} , {(cosine_sim_numbers * cosine_numbers_coef)[idx]:.2f} , {(cosine_sim_phrase * cosine_phrase_coef)[idx]:.2f}")

Persian number ۷ is NOT in the vocabulary.
Persian number ۷ is NOT in the vocabulary.
Persian number ۷ is NOT in the vocabulary.
Top 3 Hybrid Results:
Chunk 1 (Score: 0.00):
قانون اساسی جمهوری اسلامی ایران مبین نهاد فرهیخت#فرهنگ اجتماعی سیاسی اقتصادی جامعه ایران اساس اصول ضوابط اسلامی انعکاس خواست قلبی امت اسلامی میباشد ماهیت انقلاب عظیم اسلامی ایران مبارزه مردم مسلمان پیروزی شعار قاطع کوبنده قشر مردم تبلور مییافت خواست اساسی طلیعه پیروزی ملت نیل میطلبد ویژگی بنیادی انقلاب نهضت ایران سده مکتبی اسلامی ملت مسلمان ایران گذر نهضت ضد استبدادی مشروطه نهضت ضد استعماری ملی نفت تجربه گرانبار دست اساسی موفقیت نهضت مکتبی نبودن مبارزات گرچه نهضت خط فکری اسلامی رهبری روحانیت مبارز اصلی اساسی دلیل مبارزات مواضع اصیل اسلامی جنبش سرعت رکود کشاند#کشان
0.00 , 0.00 , 0.00
Chunk 70 (Score: 0.00):
اصل ۱۱۷ رئیس جمهور اکثریت مطلق آراء شرکتکنندگان انتخاب میشود هرگاه هیچیک نامزد اکثریت بدست نیاورد روز جمعه هفته رأی میشود نفر نامزد آراء داشتهاند شرکت میکنند نامزد دارنده آراء شرکت انتخابات منصرف بقیه نفر رأی داش

In [ ]:
df = pd.DataFrame()
df = pd.read_csv('/content/drive/My Drive/api_key.csv')

In [ ]:
from openai import OpenAI

client = OpenAI(
  base_url="https://openrouter.ai/api/v1",
  api_key=df.loc[0, 'api_key'],
)

completion = client.chat.completions.create(
  extra_headers={
  },
  model="GPT-4o-mini",
  messages=[
    {
      "role": "user",
      "content": f"{top_3[0]}\n{top_3[1]}\n{top_3[2]}\nطبق متن های بالا به طور خلاصه(در حد یک پاراگراف) به این سوال جواب بده و اشاره ای به کلمه پاراگراف نکن:{query}\n"
    }
  ]
)
print(completion.choices[0].message.content)

وظایف قوه قضائیه شامل رسیدگی و صدور حکم در تظلمات و شکایات، احیای حقوق عامه و گسترش عدل و آزادی‌های مشروع، نظارت بر اجرای قوانین، کشف جرم و تعقیب و مجازات مجرمین، و پیشگیری از وقوع جرم و اصلاح مجرمین است. همچنین، این قوه مسئول ایجاد تشکیلات مناسب در دادگستری، تهیه لوایح قضایی، استخدام و عزل قضات و سایر امور اداری مرتبط با قوه قضائیه می‌باشد.
